In [1]:
# Load dependencies
import pandas as pd
import numpy as np
import gdal
from scipy.stats import gmean
import sys
sys.path.insert(0,'../../statistics_helper/')
from fraction_helper import *
from CI_helper import *
from excel_utils import *
pd.options.display.float_format = '{:,.1e}'.format

# Estimating the biomass of marine fungi
To estimate the total biomass of marine fungi, we consider different locations in which marine fungi might reside, and estimate the total biomass of fungi in each region. The main regions we consider are epipelagic and deep-sea planktonic fungi, and particle-attached fungi.

## Epipelagic fungi
To estimate the total biomass of epipelagic free-living fungi, we rely on studies using two independent methods: qPCR and direct counts of the concentration of fungi.

### qPCR-based method
Our qPCR-based estimate measured the ratio between DNA copy numbers for bacteria and fugni in the West Pacific Warm Pool ([Wang et al.](https://doi.org/10.1371/journal.pone.0101523)). Here is a sample of the data:

In [6]:
qPCR_data = pd.read_excel('marine_fungi_data.xlsx','Wang',skiprows=1)
qPCR_data.head()

,Station,Depth [m],Basidiomycota [ng µl^-1],Ascomycota [ng µl^-1],Bacteria [ng µl^-1]
0,S2,5,1.5e-01,3.8e-03,2.8e+00
1,S2,25,1.0e-01,3.5e-03,2.5e+00
2,S2,50,1.7e-01,1.3e-03,2.5e+00
3,S2,75,1.8e-01,2.9e-03,9.8e-01
4,S2,100,2.5e-01,1.3e-03,1.6e+00


We calculate the ratio of fungal DNA copy number to bacterial DNA copy number:

In [46]:
# Calculate the total DNA copy number of fungi
fungal_DNA = qPCR_data['Basidiomycota [ng µl^-1]']+qPCR_data['Ascomycota [ng µl^-1]']

# Calculate the mean ratio of fungal DNA copy number and bacterial DNA copy number
qPCR_fungal_fraction = (fungal_DNA/qPCR_data['Bacteria [ng µl^-1]']).mean()

print('The ratio of fungal DNA copy number and bacterial DNA copy number is ≈%.0f' %(qPCR_fungal_fraction*100) + '%')

The ratio of fungal DNA copy number and bacterial DNA copy number is ≈12%


### Direct count method
As an independent method for estimating the total biomass of marine fungi in the epipelagic layer, we use a study which measured the carbon concentration of fungi in the epipelagic layer in upwelling ecosystem off Chile using direct counts ([Gutiérrez et al.]( https://doi.org/10.1007/s00227-010-1552-z)). We calculate the average concentration of fungal carbon in relation to the carbon concentration of prokaryotes in the same site:

In [45]:
# Load data on direct counts of fungal carbon concentration
direct_data = pd.read_excel('marine_fungi_data.xlsx','Gutiérrez',skiprows=1)

# Calculate the mean fungal carbon concentration
mean_fungal_conc = direct_data['Fungi carbon concentration [µg C L^-1]'].mean()

# Calculate the mean carbon concentration of prokaryotes
mean_prok_conc = direct_data['Prokaryote carbon concentration [µg C L^-1]'].mean()

direct_fungal_fraction = mean_fungal_conc/mean_prok_conc

print('The ratio of fungal carbon and bacterial carbon is ≈%.0f' %(direct_fungal_fraction*100) + '%')

The ratio of fungal DNA copy number and bacterial DNA copy number is ≈27%


As our best estimate for the ratio of fungal and prokaryote carbon, we use the geometric mean of the ratios estimated based on qPCR and direct counts:

In [47]:
best_fungal_fraction = gmean([qPCR_fungal_fraction,direct_fungal_fraction])
print('The ratio of fungal carbon and bacterial carbon is ≈%.0f' %(best_fungal_fraction*100) + '%')

The ratio of fungal carbon and bacterial carbon is ≈18%


To estimate the total biomass of fungi using qPCR, we rely on our estimate from the total biomass of bacteria and archaea in the top 200 meters, which we estimate in the marine bacteria and archaea section:

In [35]:
# Load total biomass of marine bacteria and archaea
marine_prok_biomass = pd.read_excel('../../bacteria_archaea/marine/marine_prok_biomass_estimate.xlsx')

# Load our estimate of the fraction of prokaryote biomass in the epipelagic layer
epi_frac = pd.read_excel('marine_fungi_data.xlsx','Bacteria biomass')

# Calculate the biomass of prokaryotes in the epipelagic realm
epi_prok_biomass = marine_prok_biomass.iloc[0:2,1].prod()*epi_frac['Value']*1e-15

print('Our estimate for the total biomass of bacteria and archaea in the epipelagic layer is ≈%.1f Gt C' %(epi_prok_biomass/1e15))

Our estimate for the total biomass of bacteria and archaea in the epipelagic layer is ≈0.4 Gt C


We estimate the total biomass of fungi in the epipelagic layer by multiplying the total biomass of prokaryotes by the ratio of DNA copy numbers we calculated based on qPCR measurements:

In [48]:
qPCR_epi_fungi = epi_prok_biomass*best_fungal_fraction

print('Our estimate for the total biomass of free-living epipelagic fungi based of qPCR is ≈%.2f Gt C' %(qPCR_epi_fungi/1e15))

Our estimate for the total biomass of free-living epipelagic fungi based of qPCR is ≈0.07 Gt C



## Deep-sea fungi


## Particle attached fungi
##